In [33]:
#imports
import spatialdata
import dask_image
import dask.array as da
import geopandas as gpd
import anndata as ad
import napari_spatialdata
import time

In [36]:
for package in [spatialdata, gpd, ad, napari_spatialdata]:
    print(f"{package.__name__}: {package.__version__}")

spatialdata: 0.2.5.post0
geopandas: 1.0.1
anndata: 0.11.1
napari_spatialdata: 0.5.3


# Create a sdata object for testing

In [2]:
#paths
path_to_mIF = "/Volumes/Extreme SSD/P26_BCCN/P26E07_E08_E09_SOPA_Gates/processed_images/991.ome.tif"
path_to_mask = "/Volumes/Extreme SSD/P26_BCCN/P26E07_E08_E09_SOPA_Gates/expanded-masks/991.tif"
path_to_shapes = "../data/geojsons/Primary/processed/20241129_1442_Primary991_harmonized.geojson"
# path_to_imaging_table = "/Volumes/Extreme SSD/P26_BCCN/P26E07_E08_E09_SOPA_Gates/adatas/20241002_1218_991_phenotyped.h5ad"
path_to_proteomics = "../data/proteomic/20241201_170620_filtered_imputed_outerjoin_mergefirst.h5ad"

In [3]:
# initiate spatialdata object
sdata = spatialdata.SpatialData()

In [4]:
#parse image
image = dask_image.imread.imread(path_to_mIF)
image = image.rechunk(chunks=(1, 5000, 5000))
parsed_image = spatialdata.models.Image2DModel.parse(image)
sdata['image'] = parsed_image

INFO     no axes information specified in the object, setting `dims` to: ('c', 'y', 'x')                           


In [5]:
#parse mask
mask = dask_image.imread.imread(path_to_mask)
mask = da.squeeze(mask)
assert mask.ndim == 2
mask = mask.rechunk(chunks=(5000, 5000))
sdata['mask'] = spatialdata.models.Labels2DModel.parse(mask)

INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                


In [6]:
# parse shapes 
gdf = gpd.read_file(path_to_shapes)
# make column matching proteomic table the index
gdf = gdf.set_index("harmonized_class")
sdata["Primary_contours"] = spatialdata.models.ShapesModel.parse(gdf)

In [7]:
#load proteomic table
adata = ad.read_h5ad(path_to_proteomics)
adata.obs["region"] = "Primary_contours"
adata.obs["region"] = adata.obs["region"].astype("category")
table = spatialdata.models.TableModel.parse(adata, region = "Primary_contours", region_key = "region", instance_key = "shape_key")
sdata["proteomics"] = table

In [8]:
def switch_adat_var_index(adata, new_index):
    """
    Created by Jose Nimo on 2023-07-01
    Lastest modified by Jose Nimo on 2024-11-16

    Description:
    Switch the index of adata.var to a new index. Useful for switching between gene names and protein names.

    Arg:
        adata: anndata object
        new_index: pandas series, new index to switch to
    Returns:
        adata: anndata object, with the new index
    """
    adata_copy = adata.copy()

    adata_copy.var[adata_copy.var.index.name] = adata_copy.var.index
    adata_copy.var.set_index(new_index, inplace=True)
    adata_copy.var.index.name = new_index
    
    return adata_copy

In [9]:
sdata['proteomics'] = switch_adat_var_index(sdata['proteomics'], "Genes")

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/spatialdata/_core/_elements.py:116: UserWarning: Key `proteomics` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


In [10]:
sdata['proteomics'].var.sort_index(inplace=True) #much easier to find a gene like this

In [11]:
#save the spatialdata object
import time
datetime = time.strftime("%Y%m%d_%H%M")
time_start = time.time()
sdata.write(f"../data/sdata/{datetime}_sdata_fullres.zarr")
time_end = time.time()
print(f"Time to save: {time_end - time_start}")

INFO     The Zarr backing store has been changed from None the new file path:                                      
         ../data/sdata/20241201_1730_sdata_fullres.zarr                                                            
Time to save: 194.9969642162323


In [15]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/1_Pipelines/openDVP/data/sdata/20241201_1730_sdata_fullres.zarr
├── Images
│     └── 'image': DataArray[cyx] (15, 44470, 73167)
├── Labels
│     └── 'mask': DataArray[yx] (44470, 73167)
├── Shapes
│     └── 'Primary_contours': GeoDataFrame shape: (347, 6) (2D shapes)
└── Tables
      └── 'proteomics': AnnData (18, 4531)
with coordinate systems:
    ▸ 'global', with elements:
        image (Images), mask (Labels), Primary_contours (Shapes)

In [21]:
# create subset of the spatialdata object
sdata_subset = spatialdata.bounding_box_query(
    sdata,
    axes = ["y", "x"],
    min_coordinate = [22886, 36125],
    max_coordinate = [43045, 61625],
    target_coordinate_system = "global",
    filter_table = False,
)

In [22]:
sdata_subset

SpatialData object
├── Images
│     └── 'image': DataArray[cyx] (15, 20159, 25500)
├── Labels
│     └── 'mask': DataArray[yx] (20159, 25500)
├── Shapes
│     └── 'Primary_contours': GeoDataFrame shape: (347, 6) (2D shapes)
└── Tables
      └── 'proteomics': AnnData (18, 4531)
with coordinate systems:
    ▸ 'global', with elements:
        image (Images), mask (Labels), Primary_contours (Shapes)

In [ ]:
interactive = napari_spatialdata.Interactive(sdata_subset)
interactive.run()

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/napari/_vispy/layers/scalar_field.py:199: UserWarning: data shape (20159, 25500) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
2024-12-01 17:43:21.590 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-12-01 17:43:21.591 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
/op

In [31]:
sdata_subset['image']

<xarray.DataArray 'image' (c: 15, y: 20159, x: 25500)> Size: 8GB
dask.array<rechunk-merge, shape=(15, 20159, 25500), dtype=uint8, chunksize=(1, 8045, 6625), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 120B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * y        (y) float64 161kB 0.5 1.5 2.5 3.5 ... 2.016e+04 2.016e+04 2.016e+04
  * x        (x) float64 204kB 0.5 1.5 2.5 3.5 ... 2.55e+04 2.55e+04 2.55e+04
Attributes:
    transform:  {'global': Sequence \n    Translation (y, x)\n        [22886....

In [26]:
datetime = time.strftime("%Y%m%d_%H%M")
sdata_subset.write(f"../data/sdata/{datetime}_sdata_subset.zarr")

ValueError: Attempt to save array to zarr with irregular chunking, please call `arr.rechunk(...)` first.